# Text-GCN Training Pipeline

This notebook walks through the complete Text-GCN training pipeline for text classification.

## Overview
Text-GCN builds a heterogeneous graph containing:
- **Document nodes**: Each text document is a node
- **Word nodes**: Each word in the vocabulary is a node
- **Edges**: Connect documents to words (TF-IDF) and words to words (PMI)

Then applies Graph Convolutional Networks for classification.

## Pipeline Steps
1. Configuration & Setup
2. Data Loading & Graph Construction
3. Model Creation
4. Training Loop
5. Evaluation
6. Results Analysis

## Memory Management
⚠️ This notebook includes explicit memory cleanup (`del` and `gc.collect()`) to handle large datasets efficiently.

---
## 1. Configuration & Setup

In [ ]:
# Import required libraries
import sys
import os
import torch
import numpy as np
from pprint import pprint
import gc
import time

# Add project root to path if needed
# project_root = os.path.dirname(os.path.abspath('__file__'))
# sys.path.insert(0, project_root)

print(f"PyTorch version: {torch.__version__}")
print(f"Python version: {sys.version}")

In [ ]:
# Import project modules
from config import FLAGS
from load_data import load_data
from saver import Saver
from model_factory import create_model
from eval import eval, MovingAverage

print("✓ All modules imported successfully")

In [ ]:
# Display configuration
print("="*60)
print("TEXT-GCN CONFIGURATION")
print("="*60)
print(f"Dataset: {FLAGS.dataset}")
print(f"Device: {FLAGS.device}")
print(f"Multi-label: {FLAGS.multi_label}")
print(f"Validation metric: {FLAGS.validation_metric}")
print(f"\nTraining Parameters:")
print(f"  - Epochs: {FLAGS.num_epochs}")
print(f"  - Learning rate: {FLAGS.lr}")
print(f"  - Window size: {FLAGS.word_window_size}")
print(f"  - Min word frequency: {FLAGS.min_word_freq}")
print(f"\nModel Parameters:")
print(f"  - Model: {FLAGS.model}")
print(f"  - Node embedding: {FLAGS.node_embd}")
print(f"  - Layer dims: {FLAGS.layer_dims}")
print(f"  - Dropout: {FLAGS.dropout}")
print("="*60)

---
## 2. Initialize Saver (for logging and model checkpoints)

In [ ]:
print("[1/5] Initializing saver...")
saver = Saver()
print(f"  ✓ Logging to: {saver.logdir}")

---
## 3. Data Loading & Graph Construction

This step:
1. Loads cleaned text documents and labels
2. Builds vocabulary (filtered by min_word_freq)
3. Constructs the heterogeneous graph:
   - Document-word edges (TF-IDF weights)
   - Word-word edges (PMI weights from co-occurrence)
4. Splits data into train/val/test sets

**Note**: This can take 5-15 minutes for large datasets (cached after first run)

In [ ]:
print("\n[2/5] Loading data...")
start_time = time.time()

train_data, val_data, test_data, raw_doc_list = load_data()

print(f"\n  ✓ Data loaded in {time.time() - start_time:.1f}s")
print(f"  ✓ Graph shape: {train_data.graph.shape}")
print(f"  ✓ Train samples: {len(train_data.node_ids)}")
print(f"  ✓ Val samples: {len(val_data.node_ids)}")
print(f"  ✓ Test samples: {len(test_data.node_ids)}")
print(f"  ✓ Vocabulary size: {len(train_data.vocab)}")
print(f"  ✓ Total nodes (docs + words): {train_data.graph.shape[0]}")
print(f"  ✓ Total edges: {train_data.graph.nnz}")

# Memory cleanup: raw_doc_list not needed after loading
del raw_doc_list, start_time
gc.collect()
print("  ✓ Memory cleanup: Removed raw_doc_list")

In [ ]:
# Inspect sample documents
print("\nSample documents (first 3):")
for i, doc_id in enumerate(train_data.node_ids[:3]):
    label = train_data.label_inds[doc_id]
    print(f"\nDoc {i+1} (node {doc_id}):")
    print(f"  Label: {label}")
    print(f"  Text: {train_data.docs_dict[doc_id][:200]}...")

In [ ]:
# Check label distribution
if FLAGS.multi_label:
    # Multi-label classification
    train_labels = np.array([train_data.label_inds[i] for i in train_data.node_ids])
    label_counts = train_labels.sum(axis=0)
    print("\nLabel distribution (multi-label):")
    for i, count in enumerate(label_counts):
        print(f"  Label {i}: {int(count)} ({100*count/len(train_data.node_ids):.1f}%)")
    
    # Cleanup
    del train_labels, label_counts
else:
    # Single-label classification
    train_labels = [train_data.label_inds[i] for i in train_data.node_ids]
    unique, counts = np.unique(train_labels, return_counts=True)
    print("\nLabel distribution (single-label):")
    for label, count in zip(unique, counts):
        print(f"  Class {label}: {count} ({100*count/len(train_labels):.1f}%)")
    
    # Cleanup
    del train_labels, unique, counts

gc.collect()

---
## 4. Model Creation

Initialize node features and create the Text-GCN model.

In [ ]:
print("\n[3/5] Creating model...")
print("  → Initializing node features...")

# Initialize node features (one-hot encoding)
train_data.init_node_feats(FLAGS.init_type, FLAGS.device)
val_data.init_node_feats(FLAGS.init_type, FLAGS.device)
test_data.init_node_feats(FLAGS.init_type, FLAGS.device)

print(f"  ✓ Feature initialization: {FLAGS.init_type}")

In [ ]:
# Create model
print("  → Building model architecture...")
model = create_model(train_data)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n  ✓ Model created")
print(f"  ✓ Total parameters: {total_params:,}")
print(f"  ✓ Trainable parameters: {trainable_params:,}")
print(f"\n  Model architecture:")
print(model)

# Cleanup
del total_params, trainable_params
gc.collect()

---
## 5. Training Loop

Train the model for the specified number of epochs with validation.

In [ ]:
print("\n[4/5] Training model...")
print(f"  → Setting up training (lr={FLAGS.lr}, epochs={FLAGS.num_epochs})...")

# Setup training
moving_avg = MovingAverage(FLAGS.validation_window_size, FLAGS.validation_metric != 'loss')
pyg_graph = train_data.get_pyg_graph(FLAGS.device)
optimizer = torch.optim.Adam(model.parameters(), lr=FLAGS.lr)

print(f"\n  {'='*56}")
print(f"  Starting training for {FLAGS.num_epochs} epochs")
print(f"  {'='*56}")

In [ ]:
# Training loop
training_history = {
    'train_loss': [],
    'val_loss': [],
    'val_metric': [],
    'epoch_times': []
}

for epoch in range(FLAGS.num_epochs):
    t_start = time.time()
    
    # Training step
    model.train()
    model.zero_grad()
    
    print(f"  Epoch {epoch+1:03d}: Forward pass...", end='', flush=True)
    t_forward_start = time.time()
    loss, preds_train = model(pyg_graph, train_data)
    t_forward = time.time() - t_forward_start
    print(f" ✓ ({t_forward:.2f}s)", flush=True)
    
    print(f"  Epoch {epoch+1:03d}: Backward pass...", end='', flush=True)
    t_backward_start = time.time()
    loss.backward()
    t_backward = time.time() - t_backward_start
    print(f" ✓ ({t_backward:.2f}s)", flush=True)
    
    print(f"  Epoch {epoch+1:03d}: Optimizer step...", end='', flush=True)
    t_opt_start = time.time()
    optimizer.step()
    t_opt = time.time() - t_opt_start
    print(f" ✓ ({t_opt:.2f}s)", flush=True)
    
    train_loss = loss.item()
    
    # Clean up training predictions (not needed)
    del preds_train
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    # Validation step
    print(f"  Epoch {epoch+1:03d}: Validation...", end='', flush=True)
    t_val_start = time.time()
    with torch.no_grad():
        val_loss, preds_val = model(pyg_graph, val_data)
        val_loss = val_loss.item()
        eval_res_val = eval(preds_val, val_data)
    t_val = time.time() - t_val_start
    print(f" ✓ ({t_val:.2f}s)", flush=True)
    
    # Clean up validation predictions
    del preds_val
    
    # Get validation metric
    metric_val = eval_res_val[FLAGS.validation_metric]
    is_best = len(moving_avg.results) == 0 or moving_avg.best_result(metric_val)
    best_marker = " 🌟 NEW BEST" if is_best else ""
    
    total_time = time.time() - t_start
    
    # Print summary
    print(f"  Epoch {epoch+1:03d} Summary: "
          f"Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, "
          f"{FLAGS.validation_metric}={metric_val:.4f}, "
          f"Total={total_time:.2f}s "
          f"(fwd:{t_forward:.1f}s + bwd:{t_backward:.1f}s + opt:{t_opt:.1f}s + val:{t_val:.1f}s)"
          f"{best_marker}")
    
    # Save history
    training_history['train_loss'].append(train_loss)
    training_history['val_loss'].append(val_loss)
    training_history['val_metric'].append(metric_val)
    training_history['epoch_times'].append(total_time)
    
    # Save best model
    if is_best:
        saver.save_trained_model(model, epoch + 1)
        print(f"    → Model saved at epoch {epoch+1}")
    
    moving_avg.add_to_moving_avg(metric_val)
    
    # Clean up eval results and timing variables
    del eval_res_val, t_forward, t_backward, t_opt, t_val, t_start
    
    # Periodic cleanup every 10 epochs
    if (epoch + 1) % 10 == 0:
        gc.collect()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        print(f"    → Memory cleanup at epoch {epoch+1}")

print(f"\n  {'='*56}")
print("  Training complete!")
print(f"  {'='*56}")

# Load best model
print("  → Loading best model...")
best_model = saver.load_trained_model(train_data)
model = best_model

# Clean up optimizer and moving average (not needed anymore)
del optimizer, moving_avg, best_model
gc.collect()
print("  ✓ Memory cleanup: Removed optimizer and training objects")

### Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Plot losses
axes[0].plot(training_history['train_loss'], label='Train Loss', marker='o')
axes[0].plot(training_history['val_loss'], label='Val Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot validation metric
axes[1].plot(training_history['val_metric'], label=FLAGS.validation_metric, marker='o', color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel(FLAGS.validation_metric)
axes[1].set_title(f'Validation {FLAGS.validation_metric}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTraining statistics:")
print(f"  Average epoch time: {np.mean(training_history['epoch_times']):.2f}s")
print(f"  Best {FLAGS.validation_metric}: {max(training_history['val_metric']):.4f}")
print(f"  Final train loss: {training_history['train_loss'][-1]:.4f}")
print(f"  Final val loss: {training_history['val_loss'][-1]:.4f}")

# Training history kept for potential future analysis
# If memory is critical, uncomment:
# del training_history
# gc.collect()

---
## 6. Test Set Evaluation

Evaluate the best model on the held-out test set.

In [ ]:
print("\n[5/5] Evaluating on test set...")
with torch.no_grad():
    test_loss, preds_test = model(pyg_graph, test_data)

print("  → Computing final test metrics...")
eval_res = eval(preds_test, test_data, True)
y_true = eval_res.pop('y_true')
y_pred = eval_res.pop('y_pred')

# Clean up test loss (not needed)
del test_loss

print("\n" + "="*60)
print("FINAL TEST RESULTS")
print("="*60)
pprint(eval_res)
print("="*60)

# Note: preds_test, y_true, y_pred kept for analysis below

### Detailed Results Analysis

In [ ]:
# Display metrics in a formatted table
import pandas as pd

metrics_df = pd.DataFrame([eval_res]).T
metrics_df.columns = ['Value']
metrics_df.index.name = 'Metric'
print("\nTest Metrics Summary:")
print(metrics_df.to_string())

# Clean up
del metrics_df
gc.collect()

In [ ]:
# For multi-label: show per-label performance
if FLAGS.multi_label:
    from sklearn.metrics import classification_report
    
    # Assuming 6 labels for Jigsaw
    label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    print("\nPer-Label Classification Report:")
    print(classification_report(y_true, y_pred, target_names=label_names, zero_division=0))
else:
    # Single-label confusion matrix
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Clean up
    del cm, disp
    gc.collect()

### ROC Curves (for multi-label classification)

In [ ]:
if FLAGS.multi_label:
    from sklearn.metrics import roc_curve, auc
    from scipy.special import expit
    
    # Get probabilities (sigmoid applied to logits)
    y_probs = expit(preds_test)  # Apply sigmoid
    
    label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    plt.figure(figsize=(10, 6))
    auc_scores = []
    for i, label in enumerate(label_names):
        fpr, tpr, _ = roc_curve(y_true[:, i], y_probs[:, i])
        roc_auc = auc(fpr, tpr)
        auc_scores.append(roc_auc)
        plt.plot(fpr, tpr, label=f'{label} (AUC = {roc_auc:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves by Label')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Mean AUC
    mean_auc = np.mean(auc_scores)
    print(f"\nMean AUC across all labels: {mean_auc:.4f}")
    
    # Clean up intermediate variables
    del fpr, tpr, roc_auc, auc_scores, mean_auc
    # Keep y_probs for inference section
else:
    y_probs = None  # Not applicable for single-label

gc.collect()

---
## 7. Inference on New Data (Optional)

Example of how to use the trained model for inference.

In [ ]:
# Example: Get predictions for specific test samples
def predict_sample(sample_idx=0):
    """Get prediction for a specific sample"""
    # Get the document node ID
    doc_node_id = test_data.node_ids[sample_idx]
    
    # Get prediction
    if FLAGS.multi_label:
        pred_probs_sample = y_probs[sample_idx]  # Already computed
        pred_labels = (pred_probs_sample > 0.5).astype(int)
        true_labels = test_data.label_inds[doc_node_id]
        
        label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        
        print(f"\nSample {sample_idx}:")
        print(f"Text: {test_data.docs_dict[doc_node_id][:200]}...")
        print(f"\nPredictions:")
        for i, label in enumerate(label_names):
            print(f"  {label:15s}: Prob={pred_probs_sample[i]:.3f}, Pred={pred_labels[i]}, True={int(true_labels[i])}")
    else:
        pred_class = y_pred[sample_idx]
        true_class = test_data.label_inds[doc_node_id]
        pred_probs_sample = torch.softmax(torch.tensor(preds_test[sample_idx]), dim=0).numpy()
        
        print(f"\nSample {sample_idx}:")
        print(f"Text: {test_data.docs_dict[doc_node_id][:200]}...")
        print(f"\nPredicted class: {pred_class}")
        print(f"True class: {true_class}")
        print(f"Class probabilities: {pred_probs_sample}")

# Test on first 3 samples
for i in range(min(3, len(test_data.node_ids))):
    predict_sample(i)
    print("-" * 60)

In [ ]:
# Clean up predictions after inference examples
del preds_test
if y_probs is not None:
    del y_probs
gc.collect()
print("✓ Cleaned up prediction tensors")

---
## 8. Model Insights

Analyze what the model learned.

In [ ]:
# Analyze most important words (highest degree in graph)
import scipy.sparse as sp

# Get word nodes (after document nodes)
num_docs = len(train_data.docs_dict)
word_start_idx = num_docs

# Calculate degree for word nodes
degrees = np.array(train_data.graph.sum(axis=0)).flatten()
word_degrees = degrees[word_start_idx:]

# Get top words by degree (most connected)
top_k = 20
top_indices = np.argsort(word_degrees)[-top_k:][::-1]
top_words = [train_data.vocab[i] for i in top_indices]
top_degrees = word_degrees[top_indices]

print(f"\nTop {top_k} most connected words in the graph:")
for i, (word, degree) in enumerate(zip(top_words, top_degrees), 1):
    print(f"{i:2d}. {word:20s} (degree: {int(degree):5d})")

# Clean up
del top_indices, top_degrees
gc.collect()

In [ ]:
# Distribution of node degrees
plt.figure(figsize=(12, 4))

doc_degrees = degrees[:num_docs]

plt.subplot(1, 2, 1)
plt.hist(doc_degrees, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Degree')
plt.ylabel('Count')
plt.title('Document Node Degree Distribution')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(word_degrees, bins=50, edgecolor='black', alpha=0.7, color='orange')
plt.xlabel('Degree')
plt.ylabel('Count')
plt.title('Word Node Degree Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nGraph statistics:")
print(f"  Document nodes: {num_docs}")
print(f"  Word nodes: {len(train_data.vocab)}")
print(f"  Total edges: {train_data.graph.nnz}")
print(f"  Avg document degree: {doc_degrees.mean():.1f}")
print(f"  Avg word degree: {word_degrees.mean():.1f}")

# Clean up degree arrays
del degrees, doc_degrees, word_degrees, num_docs, word_start_idx
gc.collect()
print("\n✓ Cleaned up degree analysis arrays")

---
## Final Cleanup (Optional)

If you're done with analysis and want to free up maximum memory.

In [ ]:
# Uncomment to clean up all major objects
# del model, train_data, val_data, test_data, pyg_graph
# del y_true, y_pred, eval_res, top_words
# if 'training_history' in locals():
#     del training_history
# gc.collect()
# torch.cuda.empty_cache() if torch.cuda.is_available() else None
# print("✓ All major objects cleaned up")

---
## Summary

This notebook demonstrated:
1. ✅ Configuration and setup
2. ✅ Data loading and graph construction
3. ✅ Model creation
4. ✅ Training loop with validation
5. ✅ Test set evaluation
6. ✅ Results visualization
7. ✅ Model insights
8. ✅ **Memory management throughout**

### Key Takeaways:
- Text-GCN builds a heterogeneous graph of documents and words
- Graph edges capture semantic relationships (TF-IDF and PMI)
- GCN layers propagate information through the graph
- The model can handle both single-label and multi-label classification
- **Explicit memory cleanup enables handling large datasets**

### Memory Management Strategy:
- ✅ Delete intermediate variables after use (`del` + `gc.collect()`)
- ✅ Clean up predictions after each phase
- ✅ Periodic cleanup during training loop
- ✅ Clear GPU cache when applicable
- ✅ Keep only essential data for subsequent cells

### Next Steps:
- Experiment with different hyperparameters (window size, min_word_freq, layer dims)
- Try different datasets
- Compare with baseline methods (BERT, Logistic Regression, etc.)
- Generate Kaggle submission (for Jigsaw dataset)